<a href="https://colab.research.google.com/github/zyz314/100-Days-Of-ML-Code/blob/master/project6_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


#indicators.py

# 导入必要的库
import pandas as pd  # 用于数据处理
import numpy as np  # 用于数值计算
import matplotlib.pyplot as plt  # 用于绘图

import util
from util import get_data  # 读取数据的辅助函数


def author():
  return "yzheng438"

# 定义技术指标函数
def MACD(df, fast_span, slow_span, signal_span):
    """
    计算移动平均线收敛背离（MACD）
    :param df: 股票数据 DataFrame
    :param fast_span: 快速EMA时间跨度
    :param slow_span: 慢速EMA时间跨度
    :param signal_span: 信号线时间跨度
    :return: macd线和信号线 (macd, signal)
    """
    ema_fast = df['price'].ewm(span=fast_span, adjust=False).mean()
    ema_slow = df['price'].ewm(span=slow_span, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_span, adjust=False).mean()
    return macd, signal

def EMA(df, span):
    """
    计算指数移动平均线（EMA）
    :param df: 股票数据 DataFrame
    :param span: 时间跨度
    :return: 指数移动平均线 ema
    """
    ema = df['price'].ewm(span=span, adjust=False).mean()
    return ema

def RSI(df, window):
    """
    计算相对强弱指数（RSI）
    :param df: 股票数据 DataFrame
    :param window: 窗口期
    :return: 相对强弱指数 rsi
    """
    delta = df['price'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def SMA(df, window):
    """
    计算简单移动平均线（SMA）
    :param df: 股票数据 DataFrame
    :param window: 窗口期
    :return: 简单移动平均线 sma
    """
    sma = df['price'].rolling(window=window).mean()
    return sma

def BollingerBands(df, window):
    """
    计算布林带
    :param df: 股票数据 DataFrame
    :param window: 窗口期
    :return: 上轨和下轨 (upper_band, lower_band)
    """
    sma = df['price'].rolling(window=window).mean()
    std = df['price'].rolling(window=window).std()
    upper_band = sma + (2 * std)
    lower_band = sma - (2 * std)
    return upper_band, lower_band

def work_all_indicators(df):
    """
    计算所有技术指标
    :param df: 股票数据 DataFrame
    :return: 所有技术指标的计算结果
    """
    macd, signal = MACD(df, fast_span=15, slow_span=30, signal_span=10)
    ema = EMA(df, span=25)
    rsi = RSI(df, window=15)
    sma = SMA(df, window=25)
    lower_band, upper_band = BollingerBands(df, window=25)
    return (macd, signal), ema, rsi, sma, (lower_band, upper_band)

def plot_indicators(df, indicators):
    """
    生成并保存所有技术指标的图表
    :param df: 股票数据 DataFrame
    :param indicators: 技术指标结果
    """
    (macd, signal), ema, rsi, sma, (lower_band, upper_band) = indicators

    # 生成并保存 MACD 指标图表
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, macd, label='MACD', color='blue')
    plt.plot(df.index, signal, label='Signal Line', color='red')
    plt.title('Moving Average Convergence Divergence (MACD)')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_1_MACD.png')
    plt.close()

    # 生成并保存 EMA 指标图表
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['price'], label='Price', color='blue')
    plt.plot(df.index, ema, label='EMA(20)', color='red')
    plt.title('Exponential Moving Average (EMA)')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_2_EMA.png')
    plt.close()

    # 生成并保存 RSI 指标图表
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, rsi, label='RSI(14)', color='red')
    plt.axhline(y=70, color='blue', linestyle='--')
    plt.axhline(y=30, color='orange', linestyle='--')
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_3_RSI.png')
    plt.close()

    # 生成并保存 SMA 指标图表
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['price'], label='Price', color='blue')
    plt.plot(df.index, sma, label='SMA(20)', color='red')
    plt.title('Simple Moving Average (SMA)')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_4_SMA.png')
    plt.close()

    # 生成并保存布林带指标图表
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['price'], label='Price', color='blue')
    plt.plot(df.index, lower_band, label='Lower Band', color='orange')
    plt.plot(df.index, upper_band, label='Upper Band', color='green')
    plt.fill_between(df.index, lower_band, upper_band, color='lightgray', alpha=0.5)
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig('Figure_5_BollingerBands.png')
    plt.close()

# 调用主函数运行指标并生成图表
if __name__ == "__main__":
    # 读取JPM股票数据，时间范围为2008年1月1日到2009年12月31日
    df = get_data(['JPM'], dates=pd.date_range('2008-01-01', '2009-12-31'))

    # 运行所有技术指标并生成相应的图表
    indicators = work_all_indicators(df)

    # 生成并保存所有技术指标的图表
    plot_indicators(df, indicators)

    # 打印结果（可选）
    (macd, signal), ema, rsi, sma, (lower_band, upper_band) = indicators
    print("MACD:\n", indicators[0][0].head())
    print("Signal:\n", indicators[0][1].head())
    print("EMA:\n", indicators[1].head())
    print("RSI:\n", indicators[2].head())
    print("SMA:\n", indicators[3].head())
    print("Bollinger Bands Upper:\n", indicators[4][1].head())
    print("Bollinger Bands Lower:\n", indicators[4][0].head())